In [ ]:
import pandas as pd

url_donnees_depistage = "https://www.data.gouv.fr/fr/datasets/r/426bab53-e3f5-4c6a-9d54-dba4442b3dbc"
df_depistage = pd.read_csv(url_donnees_depistage, sep=";")

#on se débarrasse des colonnes inutiles et qu'on doit recalculer
df_depistage = df_depistage.drop(columns=['cl_age90',"Ti","Tp","Td"])
df_depistage = df_depistage.rename(columns = {'pop':'population','P':'nb patients positifs',
'T':'nb patients testes'})

#conversion des valeurs afin de pouvoir faire des calculs
df_depistage["population"] = df_depistage["population"].str.replace(',','.').astype(float)
df_depistage["nb patients positifs"] = df_depistage["nb patients positifs"].str.replace(',','.').astype(float)
df_depistage["nb patients testes"] = df_depistage["nb patients testes"].str.replace(',','.').astype(float)

df_depistage["dep"] = df_depistage['dep'].astype(str)
df_depistage["mois"] = df_depistage["jour"].astype(str).str[5:7]
df_depistage["annee"] = df_depistage["jour"].astype(str).str[:4]

df_depistage["id"] = df_depistage["dep"] + "_" + df_depistage["annee"] + "_" + df_depistage["mois"]


In [21]:
df_depistage = df_depistage.groupby("id").agg({"dep":"first","mois":"first","annee":"first","population":"mean","nb patients positifs":"sum",
"nb patients testes":"sum"})

#définition de taux en pourcentage
df_depistage['taux de positivite'] = df_depistage['nb patients positifs']*100/df_depistage['nb patients testes']
df_depistage['taux de depistage'] = df_depistage['nb patients testes']*100/df_depistage['population']
df_depistage["taux d'incidence"] = df_depistage['nb patients positifs']*100/df_depistage['population']

df_depistage.head(10)

,dep,mois,annee,population,nb patients positifs,nb patients testes,taux de positivite,taux de depistage,taux d'incidence
id,,,,,,,,,
01_2020_05,01,05,2020,656955.0,83.0,5986.0,1.386569,0.911174,0.012634
01_2020_06,01,06,2020,656955.0,102.0,8499.0,1.200141,1.293696,0.015526
01_2020_07,01,07,2020,656955.0,150.0,15984.0,0.938438,2.433043,0.022833
01_2020_08,01,08,2020,656955.0,765.0,23369.0,3.273568,3.557169,0.116446
01_2020_09,01,09,2020,656955.0,2348.0,43071.0,5.451464,6.556157,0.357407
01_2020_10,01,10,2020,656955.0,14920.0,75977.0,19.637522,11.565023,2.271084
01_2020_11,01,11,2020,656955.0,11919.0,69038.0,17.264405,10.508787,1.814280
01_2020_12,01,12,2020,656955.0,5010.0,91603.0,5.469253,13.943573,0.762609
01_2021_01,01,01,2021,656955.0,5817.0,63391.0,9.176382,9.649215,0.885449


In [22]:
df_test = df_depistage.set_index("dep")
df_test.head(10)

,mois,annee,population,nb patients positifs,nb patients testes,taux de positivite,taux de depistage,taux d'incidence
dep,,,,,,,,
01,05,2020,656955.0,83.0,5986.0,1.386569,0.911174,0.012634
01,06,2020,656955.0,102.0,8499.0,1.200141,1.293696,0.015526
01,07,2020,656955.0,150.0,15984.0,0.938438,2.433043,0.022833
01,08,2020,656955.0,765.0,23369.0,3.273568,3.557169,0.116446
01,09,2020,656955.0,2348.0,43071.0,5.451464,6.556157,0.357407
01,10,2020,656955.0,14920.0,75977.0,19.637522,11.565023,2.271084
01,11,2020,656955.0,11919.0,69038.0,17.264405,10.508787,1.814280
01,12,2020,656955.0,5010.0,91603.0,5.469253,13.943573,0.762609
01,01,2021,656955.0,5817.0,63391.0,9.176382,9.649215,0.885449
